In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import importlib
import data_cleaning as dc
%matplotlib inline


Tables of interest:
* imdb.title.basics  (for genres)
* imdb.title.ratings (for average ratings)
* bom.movie_gross (for studios)
* tn.movie_budgets (for budgets and worldwide gross)

## Import tables of interest

function `df_from_movie_csv` loads csv.gz by tablename into dataframe, setting index, converting date fields to date time, converting dollar fields to numeric.

In [115]:
importlib.reload(dc)
import_tables = ['imdb.title.basics','imdb.title.ratings','bom.movie_gross','tn.movie_budgets']
dfs = {}
for table_name in import_tables:
    dfs[table_name] = dc.df_from_movie_csv(table_name)

function `clean_movie_df` adds a `year` column if appropriate, filters results to 2010-2018.

In [ ]:
for table_name in import_tables:
    dfs[table_name] = dc.clean_movie_df(dfs[table_name],table_name)

## Join tables

### IMDB

Join the two `imdb` dataframes. They join directly on index, no error-checking required.

In [83]:
imdb_title_ratings_df = dfs['imdb.title.basics'].join(dfs['imdb.title.ratings'])

In [84]:
imdb_title_ratings_df.size

1023008

There are a lot of movies with the same `primary_title`. Dedupe by restricting to the entry with the highest `numvotes`.

In [85]:
imdb_title_ratings_df = dc.select_max_rows_on_key_column(imdb_title_ratings_df, 
                                                         key_column='primary_title', 
                                                         max_column='numvotes')

In [86]:
imdb_title_ratings_df.shape[0]

136071

### Box Office Mojo

In [116]:
bom_df = dfs['bom.movie_gross']
bom_df = dc.select_max_rows_on_key_column(bom_df,key_column='title',max_column='domestic_gross')
bom_df.shape[0]

3386

Join the `imdb` dataframe with the `bom` dataframe on (English-language) movie title. This misses some big movies (title conventions are not identical).

In [88]:
imdb_bom_df = dc.join_dfs_on_key_col(imdb_title_ratings_df,bom_df,
                                     left_on='primary_title',right_on='title')
imdb_bom_df.shape[0]

2605

Our matching by title has a success rate of 2605/3386 (77%).

### The Numbers

In [112]:
tn_df = dfs['tn.movie_budgets']
dc.date_to_year(tn_df,'release_date')
tn_df = tn_df.query('2009 < year < 2019')
tn_df.head()

,release_date,movie,production_budget,domestic_gross,worldwide_gross,year,foreign_gross
id,,,,,,,
2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,2011,8.046000e+08
4,2015-05-01,Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,2015,9.440081e+08
5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09,2017,6.965404e+08
6,2015-12-18,Star Wars Ep. VII: The Force Awakens,306000000.0,936662225.0,2.053311e+09,2015,1.116649e+09
7,2018-04-27,Avengers: Infinity War,300000000.0,678815482.0,2.048134e+09,2018,1.369319e+09


Join the `imdb_bom` dataframe with the `tn` dataframe on title and release year. 

In [103]:
df = imdb_bom_df.drop(columns=['domestic_gross','foreign_gross']).reset_index() \
                  .merge(tn_df.rename(columns={'movie':'primary_title'}),
                         how='inner',
                         on=['primary_title','year']) \
                  .set_index('tconst')

In [101]:
df.shape[0]

1135

In [104]:
df.head()

,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,studio,year,release_date,production_budget,domestic_gross,worldwide_gross,foreign_gross
tconst,,,,,,,,,,,,,,
tt1179933,10 Cloverfield Lane,10 Cloverfield Lane,2016,103.0,"Drama,Horror,Mystery",7.2,260383.0,Par.,2016,2016-03-11,5000000.0,72082999.0,108286422.0,36203423.0
tt1413492,12 Strong,12 Strong,2018,130.0,"Action,Drama,History",6.6,50155.0,WB,2018,2018-01-19,35000000.0,45819713.0,71118378.0,25298665.0
tt2024544,12 Years a Slave,12 Years a Slave,2013,134.0,"Biography,Drama,History",8.1,577301.0,FoxS,2013,2013-10-18,20000000.0,56671993.0,181025343.0,124353350.0
tt1542344,127 Hours,127 Hours,2010,94.0,"Adventure,Biography,Drama",7.6,323949.0,FoxS,2010,2010-11-05,18000000.0,18335230.0,60217171.0,41881941.0
tt2059171,13 Sins,13 Sins,2014,93.0,"Horror,Thriller",6.3,29550.0,RTWC,2014,2014-04-18,4000000.0,9134.0,47552.0,38418.0
